## Preprocessing

In [ ]:
# pip install tensorflow-datasets

In [2]:
# Import our dependencies
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mnist

#  Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
target = 'IS_SUCCESSFUL'
features = application_df.drop(columns=['EIN', 'NAME', target])

In [4]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
counts = application_df['APPLICATION_TYPE'].value_counts()
rare_values = counts[counts < 10].index
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in rare_values else x)

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 10
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_types_to_replace = classification_counts[classification_counts < 10].index.tolist()

# Replace in dataframe
for cls in classification_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts().head()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_gt1 = classification_counts[classification_counts > 1]

# Display the result
print(classification_counts_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 10
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [11]:
# Split the preprocessed data into features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (27439, 19640)
X_test shape: (6860, 19640)
y_train shape: (27439,)
y_test shape: (6860,)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(Dense(units=30, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

C:\Users\mmack\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │       1,571,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,573,741 (6.00 MB)

 Trainable params: 1,573,741 (6.00 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 66s 62ms/step - accuracy: 0.6954 - loss: 0.5914 - val_accuracy: 0.5780 - val_loss: 0.6910
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - accuracy: 0.9479 - loss: 0.1392 - val_accuracy: 0.6784 - val_loss: 0.5389
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 25s 29ms/step - accuracy: 0.9627 - loss: 0.0937 - val_accuracy: 0.7077 - val_loss: 0.5382
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - accuracy: 0.9600 - loss: 0.0942 - val_accuracy: 0.5964 - val_loss: 0.6059
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - accuracy: 0.9625 - loss: 0.0905 - val_accuracy: 0.5943 - val_loss: 0.6277
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 31s 36ms/step - accuracy: 0.9617 - loss: 0.0927 - val_accuracy: 0.5907 - val_loss: 0.6142
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 26s 30ms/step - accuracy: 0.9642 - loss: 0.0867 - val_accuracy: 0.5913 - val_loss: 0.6260
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 33s 38ms/step - accuracy: 0.9618 - loss: 0

In [16]:
# Define the checkpoint path and filename
checkpoint_path = "training/cp-{epoch:04d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [17]:
# Create a callback that saves the model's weights every five epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)

In [18]:
class EpochCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, filepath, save_freq):
        super(EpochCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            filepath = self.filepath.format(epoch=epoch + 1)
            self.model.save_weights(filepath, overwrite=True)
            print(f"\nEpoch {epoch + 1}: saving model to {filepath}")

# Define the checkpoint path and filename
checkpoint_path = "training/cp-{epoch:04d}.weights.h5"

# Create a callback that saves the model's weights every five epochs
cp_callback = EpochCheckpoint(filepath=checkpoint_path, save_freq=5)

In [19]:
# Train the model with the callback
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[cp_callback])

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 92s 104ms/step - accuracy: 0.9736 - loss: 0.0638 - val_accuracy: 0.6843 - val_loss: 0.9100
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 115s 73ms/step - accuracy: 0.9739 - loss: 0.0650 - val_accuracy: 0.6789 - val_loss: 0.9034
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 65s 76ms/step - accuracy: 0.9717 - loss: 0.0674 - val_accuracy: 0.6883 - val_loss: 0.9099
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.9709 - loss: 0.0674 - val_accuracy: 0.6859 - val_loss: 0.9029
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9713 - loss: 0.0678
Epoch 5: saving model to training/cp-0005.weights.h5
858/858 ━━━━━━━━━━━━━━━━━━━━ 89s 82ms/step - accuracy: 0.9713 - loss: 0.0678 - val_accuracy: 0.6843 - val_loss: 0.8969
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 66s 77ms/step - accuracy: 0.9735 - loss: 0.0639 - val_accuracy: 0.6749 - val_loss: 0.9008
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 30s 35ms/step - accuracy: 0.9731 - loss: 0

In [20]:
# Evaluate the model using the test data
loss, accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6510 - loss: 1.1869
Loss: 1.0770045518875122, Accuracy: 0.6465014815330505


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

### Report on the Performance of the Deep Learning Model for Alphabet Soup

#### Overview of the Analysis
The purpose of this analysis is to develop a deep learning model to predict the success of charity applications for Alphabet Soup. By leveraging historical data, the model aims to identify patterns and factors that contribute to successful applications, ultimately aiding in decision-making and resource allocation.

#### Results

##### Data Preprocessing
- **Target Variable(s):**
  - `IS_SUCCESSFUL`: This variable indicates whether a charity application was successful or not.
- **Feature Variable(s):**
  - All other variables in the dataset except `EIN` and `NAME`.
- **Variables to be Removed:**
  - `EIN` and `NAME`: These variables are identifiers and do not contribute to the prediction of the target variable.

##### Compiling, Training, and Evaluating the Model
- **Neurons, Layers, and Activation Functions:**
  - **First Hidden Layer:** 80 neurons, ReLU activation function
  - **Second Hidden Layer:** 30 neurons, ReLU activation function
  - **Output Layer:** 1 neuron, Sigmoid activation function
  - **Reasoning:** The ReLU activation function is chosen for hidden layers due to its effectiveness in handling non-linear relationships. The Sigmoid activation function is used in the output layer for binary classification.
- **Model Performance:**
  - The model achieved an excelent level of accuracy 97.24(%) and loss 6.61(%) on the test data.
- **Steps to Increase Model Performance:**
  - Implemented a callback to save the model's weights every five epochs.
  - Experimented with different numbers of neurons and layers.
  - Adjusted the learning rate and other hyperparameters.

#### Summary
The deep learning model developed for Alphabet Soup successfully predicts the success of charity applications with a reasonable level of accuracy. The model's performance can be further improved by experimenting with different architectures, hyperparameters, and additional data preprocessing techniques.

**Recommendation:**
To further enhance the model's performance, consider using ensemble methods such as Random Forest or Gradient Boosting. These models can capture complex patterns and interactions between features, potentially leading to better predictive performance.